# Labelme转YOLO-单个文件

同济子豪兄 2023-4-16

## 导入工具包

In [1]:
import os
import json
import numpy as np

## 数据集类别信息

In [10]:
classes = {
    'Angle':0,
    'Ruler':1,
    'Triangle_30':2,
    'Triangle_45':3
}

In [11]:
list(classes.keys())

['Angle', 'Ruler', 'Triangle_30', 'Triangle_45']

## 生成`classes.txt`文件

In [12]:
with open('classes.txt', 'w', encoding='utf-8') as f:
    for each in list(classes.keys()):
        f.write(each + '\n')

## 载入一个labelme格式的json标注文件

In [13]:
labelme_path = '1.json'

In [14]:
with open(labelme_path, 'r', encoding='utf-8') as f:
    labelme = json.load(f)

In [15]:
labelme.keys()

dict_keys(['version', 'flags', 'shapes', 'imagePath', 'imageData', 'imageHeight', 'imageWidth'])

### 元数据 

In [16]:
labelme['version']

'5.1.1'

In [17]:
labelme['flags']

{}

In [18]:
# 图像文件名
labelme['imagePath']

'1.jpg'

In [19]:
labelme['imageData']

In [20]:
# 图像高度
labelme['imageHeight']

2736

In [21]:
# 图像宽度
labelme['imageWidth']

3648

### 标注信息

In [22]:
labelme['shapes']

[{'label': 'Triangle_30',
  'points': [[172.52941176470614, 100.35294117647076],
   [2054.8823529411766, 976.8235294117648]],
  'group_id': None,
  'shape_type': 'rectangle',
  'flags': {}},
 {'label': 'Triangle_45',
  'points': [[2422.529411764706, 432.70588235294133],
   [3569.5882352941176, 1673.8823529411766]],
  'group_id': None,
  'shape_type': 'rectangle',
  'flags': {}},
 {'label': 'Angle',
  'points': [[1128.4117647058827, 1000.3529411764706],
   [2510.764705882353, 1844.4705882352944]],
  'group_id': None,
  'shape_type': 'rectangle',
  'flags': {}},
 {'label': 'Ruler',
  'points': [[290.17647058823553, 2029.764705882353],
   [2510.764705882353, 2626.8235294117644]],
  'group_id': None,
  'shape_type': 'rectangle',
  'flags': {}}]

## 生成YOLO格式的标注文件

In [23]:
img_width = labelme['imageWidth']   # 图像宽度
img_height = labelme['imageHeight'] # 图像高度

# 生成 YOLO 格式的 txt 文件
suffix = labelme_path.split('.')[-2]
yolo_txt_path = suffix + '.txt'

with open(yolo_txt_path, 'w', encoding='utf-8') as f:
    for each_ann in labelme['shapes']: # 遍历每个框

        if each_ann['shape_type'] == 'rectangle': # 筛选出框

            # 获取类别 ID
            bbox_class_id = classes[each_ann['label']]

            # 左上角和右下角的 XY 像素坐标
            bbox_top_left_x = int(min(each_ann['points'][0][0], each_ann['points'][1][0]))
            bbox_bottom_right_x = int(max(each_ann['points'][0][0], each_ann['points'][1][0]))
            bbox_top_left_y = int(min(each_ann['points'][0][1], each_ann['points'][1][1]))
            bbox_bottom_right_y = int(max(each_ann['points'][0][1], each_ann['points'][1][1]))

            # 框中心点的 XY 像素坐标
            bbox_center_x = int((bbox_top_left_x + bbox_bottom_right_x) / 2)
            bbox_center_y = int((bbox_top_left_y + bbox_bottom_right_y) / 2)

            # 框宽度
            bbox_width = bbox_bottom_right_x - bbox_top_left_x

            # 框高度
            bbox_height = bbox_bottom_right_y - bbox_top_left_y

            # 框中心点归一化坐标
            bbox_center_x_norm = bbox_center_x / img_width
            bbox_center_y_norm = bbox_center_y / img_height

            # 框归一化宽度
            bbox_width_norm = bbox_width / img_width
            # 框归一化高度
            bbox_height_norm = bbox_height / img_height

            # 生成 YOLO 格式的一行标注，指定保留小数点后几位
            bbox_yolo_str = '{} {:.4f} {:.4f} {:.4f} {:.4f}'.format(bbox_class_id, bbox_center_x_norm, bbox_center_y_norm, bbox_width_norm, bbox_height_norm)
            # 写入 txt 文件中
            f.write(bbox_yolo_str + '\n')
            
print('{} --> {} 转换完成'.format(labelme_path, yolo_txt_path))

1.json --> 1.txt 转换完成
